In [1]:
import re
import time
import pickle
import numpy as np

from neural_decoder.neural_decoder_trainer_mamba import loadModel, getDatasetLoaders
from neural_decoder.model import GRUDecoder, MambaDecoder
from neural_decoder.dataset import SpeechDataset
import neuralDecoder.utils.lmDecoderUtils as lmDecoderUtils

from edit_distance import SequenceMatcher
import torch

from tqdm import tqdm

In [4]:
mamba_outputs = pickle.load(open('235_mamba_outputs_competition.pkl', 'rb'))

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

llm, llm_tokenizer = lmDecoderUtils.build_opt(
    cacheDir='/scratch/users/hdlee/speech_bci/language_models/llms', device="auto", load_in_8bit=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#loads the language model, could take a while and requires ~60 GB of memory
lmDir = '/scratch/users/hdlee/speech_bci/language_models/speech_5gram/lang_test' 
# lmDir = '/scratch/users/hdlee/speech_bci/language_models/languageModel'
ngramDecoder = lmDecoderUtils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.5,
    nbest=100,
    beam=18
)

I0425 13:04:54.852828 76490 brain_speech_decoder.h:52] Reading fst /scratch/users/hdlee/speech_bci/language_models/speech_5gram/lang_test/TLG.fst
I0425 13:09:18.055339 76490 brain_speech_decoder.h:58] Reading lm fst /scratch/users/hdlee/speech_bci/language_models/speech_5gram/lang_test/G.fst
I0425 13:10:12.606490 76490 brain_speech_decoder.h:70] Reading rescore fst /scratch/users/hdlee/speech_bci/language_models/speech_5gram/lang_test/G_no_prune.fst
I0425 13:24:23.419416 76490 brain_speech_decoder.h:81] Reading symbol table /scratch/users/hdlee/speech_bci/language_models/speech_5gram/lang_test/words.txt


In [ ]:
# # LM decoding hyperparameters
# acoustic_scale = 0.5
# blank_penalty = np.log(2)
# llm_weight = 0.5

# llm_outputs = []
# # Generate nbest outputs from an LM
# start_t = time.time()
# nbest_outputs = []

# j = 0
# logits = mamba_outputs["logits"][j]
# logits = np.concatenate(
#     [logits[:, 1:], logits[:, 0:1]], axis=-1
# )  # Blank is last token
# # print(logits.shape)

# logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
# print(logits.shape)

# logPriors = np.zeros([1, logits[0].shape[1]])

# import lm_decoder

# lm_decoder.DecodeNumpy(ngramDecoder, logits[0], logPriors, blank_penalty)

# ngramDecoder.FinishDecoding()

# # ngramDecoder.Rescore() #ERROR

In [ ]:
# LM decoding hyperparameters
acoustic_scale = 0.5
blank_penalty = np.log(7)
llm_weight = 0.5

llm_outputs = []
# Generate nbest outputs from an LM
start_t = time.time()
nbest_outputs = []
for j in tqdm(range(len(mamba_outputs["logits"]))):
    logits = mamba_outputs["logits"][j]
    logits = np.concatenate(
        [logits[:, 1:], logits[:, 0:1]], axis=-1
    )  # Blank is last token
    logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
    nbest = lmDecoderUtils.lm_decode(
        ngramDecoder,
        logits[0],
        blankPenalty=blank_penalty,
        returnNBest=True,
        rescore=False,#True,
    )
    nbest_outputs.append(nbest)
time_per_sample = (time.time() - start_t) / len(mamba_outputs["logits"])
print(f"decoding took {time_per_sample} seconds per sample")

 67%|██████▋   | 805/1200 [09:24<03:08,  2.10it/s]

In [ ]:
for i in range(len(mamba_outputs["transcriptions"])):
    new_trans = [ord(c) for c in mamba_outputs["transcriptions"][i]] + [0]
    mamba_outputs["transcriptions"][i] = np.array(new_trans)

In [18]:
# k = 5
# topk_best_outputs = [output[:k] for output in nbest_outputs]

In [19]:
# # Rescore nbest outputs with LLM
# start_t = time.time()
# llm_out = lmDecoderUtils.cer_with_gpt2_decoder(
#     llm,
#     llm_tokenizer,
#     topk_best_outputs[:],
#     acoustic_scale,
#     mamba_outputs,
#     outputType="speech_sil",
#     returnCI=True,
#     lengthPenalty=0,
#     alpha=llm_weight,
# )
# time_per_sample = (time.time() - start_t) / len(logits)
# print(f"LLM decoding took {time_per_sample} seconds per sample")

  0%|          | 0/1200 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
pickle.dump((nbest_outputs, mamba_outputs), open('./235_inputs_for_decoder_5gram.pkl', 'wb'))

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

llm, llm_tokenizer = lmDecoderUtils.build_opt(
    cacheDir='/scratch/users/hdlee/speech_bci/language_models/llms', device="auto", load_in_8bit=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# LM decoding hyperparameters
acoustic_scale = 0.5
blank_penalty = np.log(7)
llm_weight = 0.5

In [4]:
nbest_outputs, mamba_outputs = pickle.load(open('./235_inputs_for_decoder.pkl', 'rb'))

In [5]:
k = 100
topk_best_outputs = [output[:k] for output in nbest_outputs]

In [6]:
# Rescore nbest outputs with LLM
start_t = time.time()
llm_out = lmDecoderUtils.cer_with_gpt2_decoder(
    llm,
    llm_tokenizer,
    topk_best_outputs[:],
    acoustic_scale,
    mamba_outputs,
    outputType="speech_sil",
    returnCI=True,
    lengthPenalty=0,
    alpha=llm_weight,
)
# time_per_sample = (time.time() - start_t) / len(logits)
# print(f"LLM decoding took {time_per_sample} seconds per sample")

  0%|          | 0/1200 [00:00<?, ?it/s]

In [7]:
print(llm_out["cer"], llm_out["wer"])
with open("/scratch/users/hdlee/speech_bci/logs/Mamba_Run2_speckled_masking_layerNorm_standardized_bidirectional_6layers/llm_out", "wb") as handle:
    pickle.dump(llm_out, handle)

(3.1315625, 3.069059895833333, 3.195) (3.067083333333333, 3.0091666666666668, 3.1258333333333335)


In [8]:
decodedTranscriptions = llm_out["decoded_transcripts"]
with open("./235_3gramLLMCompetitionSubmission.txt", "w") as f:
    for x in range(len(decodedTranscriptions)):
        f.write(decodedTranscriptions[x] + "\n")

In [7]:
def _extract_transcriptions(inferenceOut):
    transcriptions = []
    for i in range(len(inferenceOut['transcriptions'])):
        endIdx = np.argwhere(inferenceOut['transcriptions'][i] == 0)[0, 0]
        trans = ''
        for c in range(endIdx):
            trans += chr(inferenceOut['transcriptions'][i][c])
        transcriptions.append(trans)

    return transcriptions

trueSentences = _extract_transcriptions(mamba_outputs)

In [8]:
len(nbest_outputs)

1200

In [ ]:
decoded_sentences = []
for output in nbest_outputs:
    decoded_sentences.append(output[0][0].strip())

In [ ]:
len(decoded_sentences)

In [ ]:
decoded_sentences[-10:]

In [ ]:
with open('testCompetitionSubmission.txt', 'w') as f:
    for x in range(len(decoded_sentences)):
        f.write(decoded_sentences[x]+'\n')

In [ ]:
!ml load cudnn/8.6.0.163
!ml load cuda/11.7.1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

modelName = 'facebook/opt-6.7b'
cacheDir = '/scratch/users/hdlee/speech_bci/language_models/llms'
device = 'auto'
load_in_8bit = False

tokenizer = AutoTokenizer.from_pretrained(modelName, cache_dir=cacheDir)
model = AutoModelForCausalLM.from_pretrained(modelName, cache_dir=cacheDir,
                                             device_map=device, load_in_8bit=load_in_8bit)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Rescore nbest outputs with LLM
start_t = time.time()
llm_out = lmDecoderUtils.cer_with_gpt2_decoder(
    llm,
    llm_tokenizer,
    nbest_outputs[:],
    acoustic_scale,
    mamba_outputs,
    outputType="speech_sil",
    returnCI=True,
    lengthPenalty=0,
    alpha=llm_weight,
)
time_per_sample = (time.time() - start_t) / len(logits)
print(f"LLM decoding took {time_per_sample} seconds per sample")

In [ ]:
print(llm_out["cer"], llm_out["wer"])
with open(input_args.modelPath + "/llm_out", "wb") as handle:
    pickle.dump(llm_out, handle)

decodedTranscriptions = llm_out["decoded_transcripts"]
with open(input_args.modelPath + "/LLMCompetitionSubmission.txt", "w") as f:
    for x in range(len(decodedTranscriptions)):
        f.write(decodedTranscriptions[x] + "\n")